In [62]:
import glob
import numpy as np
import matplotlib.pyplot as plt

import os 
import imageio
import scipy.io as spio
import cv2
from numpy import sqrt, pi, exp, transpose, matmul
from numpy.linalg import det, inv
from scipy.stats import norm 
from scipy.stats import multivariate_normal
import time
import sys

%matplotlib inline
plt.style.use('default')

In [63]:
# training

# load the original picture 
files_ori = glob.glob("apples/*.jpg")
ColorImgs = []
for myFile in files_ori:
    im = plt.imread(myFile)
    ColorImgs.append(im)

# load the mask
files_mask = glob.glob("apples/*.png")
MaskImgs = []
for myFile in files_mask:
    im = plt.imread(myFile)
    gray_image = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    MaskImgs.append(gray_image)

nIter = 70

# MixGaussian Code from practical C
def getMixGaussLogLike(data, mixGaussEst): 
    """
    Calculate the log likelihood for the whole dataset under a mixture of Gaussians model.
    
    Keyword arguments:
    data -- d by n matrix containing data points.
    mixGaussEst -- dict containing the mixture of gaussians parameters.

    Returns: 
    logLike -- scalar containing the log likelihood.
    
    """
    
    data = np.atleast_2d(data)                                                                         
    # find total number of data items                                                                  
    nDims, nData = data.shape                                                                          
    
    # initialize log likelihoods                                                                       
    logLike = 0;                                                                                       
                                                                                                       
    # run through each data item                                                                       
    for cData in range(nData):                                                                         
        thisData = data[:, cData]                                                                      
        # TO DO - calculate likelihood of this data point under mixture of                         
        # Gaussians model. Replace this                                                                
        like = 0
        for k in range(mixGaussEst['k']):
            # Multivariate normal distribution PDF
            cov_k = mixGaussEst['cov'][:,:,k]
            mean_k = mixGaussEst['mean'][:,k]
            weight_k = mixGaussEst['weight'][k]
            centered_data_k = thisData - mean_k
            
            # Calculate likelihood for the k-th component
            like_k = weight_k * (1 / np.sqrt(((2 * np.pi) ** mixGaussEst['d']) * np.linalg.det(cov_k))) * \
                     np.exp(-0.5 * np.dot(np.dot((centered_data_k).T, np.linalg.inv(cov_k)), (centered_data_k)))
            like_k = max(like_k, 1e-10)

            # Accumulate likelihood over all components
            like += like_k
        # add to total log like                                                                        
        logLike = logLike + np.log(like)                                                               
                                                                                                       
    return  logLike.item()                                                                       
                                                                                                       
def fitMixGauss(data, k):
    """
    Estimate a k MoG model that would fit the data. Incremently plots the outcome.
               
    
    Keyword arguments:
    data -- d by n matrix containing data points.
    k -- scalar representing the number of gaussians to use in the MoG model.
    
    Returns: 
    mixGaussEst -- dict containing the estimated MoG parameters.
    
    """
    
    #     MAIN E-M ROUTINE  
    #     In the E-M algorithm, we calculate a complete posterior distribution over                                  
    #     the (nData) hidden variables in the E-Step.  
    #     In the M-Step, we update the parameters of the Gaussians (mean, cov, w).   
    
    nDims, nData = data.shape


    postHidden = np.zeros(shape=(k, nData))

    # we will initialize the values to random values
    mixGaussEst = dict()
    mixGaussEst['d'] = nDims
    mixGaussEst['k'] = k
    mixGaussEst['weight'] = (1 / k) * np.ones(shape=(k))
    mixGaussEst['mean'] = 2 * np.random.randn(nDims, k)
    mixGaussEst['cov'] = np.zeros(shape=(nDims, nDims, k))
    # for cGauss in range(k):
    #     mixGaussEst['cov'][:, :, cGauss] = 2.5 + 1.5 * np.random.uniform() * np.eye(nDims)     
    for cGauss in range(k):
        mixGaussEst['cov'][:, :, cGauss] = 2.5 + 1.5 * np.random.uniform() * np.eye(nDims) + np.eye(nDims) * 0.001

    # calculate current likelihood
    # TO DO - fill in this routine
    logLike = getMixGaussLogLike(data, mixGaussEst)
    print('Log Likelihood Iter 0 : {:4.3f}\n'.format(logLike))


    for cIter in range(nIter):

        # ===================== =====================
        # Expectation step
        # ===================== =====================

        for cData in range(nData):
            # TO DO (g) : fill in column of 'hidden' - calculate posterior probability that
            # this data point came from each of the Gaussians
            # replace this:
            x = data[:, cData]
            #print(x.shape)
            for cGauss in range(k):
                kthMean = mixGaussEst['mean'][:, cGauss]
                kthCov = mixGaussEst['cov'][:, :, cGauss]
                kthWeight = mixGaussEst['weight'][cGauss]
                
                # Calculate the posterior probability using Bayes' rule
                postHidden[cGauss, cData] = kthWeight * multivariate_normal.pdf(x, kthMean, kthCov)
                
            # Normalize the responsibilities to make them probabilities
            postSum = np.sum(postHidden[:, cData])
            if postSum == 0:
                postHidden[:, cData] = 1.0 / k
            else:
                postHidden[:, cData] /= postSum
            #postHidden[: , cData] /= np.sum(postHidden[:, cData])
            

        # ===================== =====================
        # Maximization Step
        # ===================== =====================
        # for each constituent Gaussian
        for cGauss in range(k):
            # TO DO (h):  Update weighting parameters mixGauss.weight based on the total
            # posterior probability associated with each Gaussian. Replace this:
            postHidden_sum = np.sum(postHidden)
            mixGaussEst['weight'][cGauss] = np.sum(postHidden[cGauss, :]) / postHidden_sum
            
            # TO DO (i):  Update mean parameters mixGauss.mean by weighted average
            # where weights are given by posterior probability associated with
            # Gaussian.  Replace this:
            mixGaussEst['mean'][:,cGauss] = np.dot(postHidden[cGauss, :], data.T) / np.sum(postHidden[cGauss, :])
            
            # TO DO (j):  Update covarance parameter based on weighted average of
            # square distance from update mean, where weights are given by
            # posterior probability associated with Gaussian
            centered_data = data - np.expand_dims(mixGaussEst['mean'][:, cGauss], axis=1)
            mixGaussEst['cov'][:, :, cGauss] = np.dot(postHidden[cGauss, :] * centered_data, centered_data.T) / np.sum(postHidden[cGauss, :])
            
            
            # draw the new solution
        print(mixGaussEst)

        # calculate the log likelihood
        logLike = getMixGaussLogLike(data, mixGaussEst)
        print('Log Likelihood After Iter {} : {:4.3f}\n'.format(cIter, logLike))


    return mixGaussEst

# generate trainng data
apple_pixels = []
nonapple_pixels = []
for color_img, mask_img in zip(ColorImgs, MaskImgs):
    # extract the pixels which belong to apple regions
    mask_img = np.where(mask_img > 0.5, 1, 0)
    apple_pixel = color_img[mask_img == 1]
    nonapple_pixel = color_img[mask_img == 0]
    apple_pixels.extend(apple_pixel)
    nonapple_pixels.extend(nonapple_pixel)

# trans trainng data to numpy array
apple_pixels = np.array(apple_pixels)
apple_pixels = apple_pixels.T
nonapple_pixels = np.array(nonapple_pixels)
nonapple_pixels = nonapple_pixels.T

print(apple_pixels.shape)
print(apple_pixels)

nGaussEst = 2
mixGaussEstApple = fitMixGauss(apple_pixels, nGaussEst)
mixGaussEstNonApple = fitMixGauss(nonapple_pixels, nGaussEst)


(3, 425694)
[[179 189 197 ...  40  39  39]
 [124 131 137 ...   0   0   1]
 [ 85  93 100 ...   0   0   0]]
Log Likelihood Iter 0 : -9505667.822

{'d': 3, 'k': 2, 'weight': array([0.50883099, 0.49116901]), 'mean': array([[141.76019944, 145.75653045],
       [114.39451972, 117.64249806],
       [ 52.48460631,  54.10843689]]), 'cov': array([[[2734.69527856, 2365.48389421],
        [2440.97226959, 2152.51611251],
        [2000.93944671, 1882.89809843]],

       [[2440.97226959, 2152.51611251],
        [5536.14519488, 5420.52366578],
        [2495.85630706, 2430.14010845]],

       [[2000.93944671, 1882.89809843],
        [2495.85630706, 2430.14010845],
        [3282.92961481, 3321.20029436]]])}
Log Likelihood After Iter 0 : -6793168.362

{'d': 3, 'k': 2, 'weight': array([0.50877033, 0.49122967]), 'mean': array([[140.22336492, 147.34774817],
       [113.0975613 , 118.98536669],
       [ 51.25101121,  55.38588023]]), 'cov': array([[[2718.86197336, 2364.22995745],
        [2412.1459893 , 2167.

In [65]:
# testing

def calculate_posterior(image_pixel, mixGaussEst):
    k = mixGaussEst['k']
    posterior = 0
    for i in range(k):
        weight = mixGaussEst['weight'][i]
        mean = mixGaussEst['mean'][:, i]
        cov = mixGaussEst['cov'][:, :, i]
        posterior += weight * multivariate_normal.pdf(image_pixel, mean, cov)
    return posterior                                     
 
# let's define priors for whether the pixel is apple or non apple
priorApple = 0.5
priorNonApple = 0.5
files_test = glob.glob("testApples/*.jpg")
TestImgs = []

for myFile in files_test:
    im = plt.imread(myFile)
    imY, imX, imZ = im.shape
    posteriorSkin = np.zeros([imY,imX])

    for cY in range(imY): 
        print('Processing Row ',cY,'\n')
        for cX in range(imX):
            #extract this pixel's data
            thisPixelData = np.double(im[cY,cX,:])
            thisPixelData = thisPixelData.flatten()
            
            #calculate likelihood of this data given skin model
            likeApple = calculate_posterior(thisPixelData,mixGaussEstApple)
            #calculate likelihood of this data given non skin model
            likeNonApple = calculate_posterior(thisPixelData,mixGaussEstNonApple)
            
            #TO DO (c):  calculate posterior probability from likelihoods and 
            #priors using BAYES rule. Replace this: 
            # threshold = 0.5
            # posteriorSkin[cY, cX] = (likeApple * priorApple) / (likeApple * priorApple + likeNonApple * priorNonApple)
            # if posteriorSkin[cY, cX] > threshold:
            #     posteriorSkin[cY, cX] = 1
            # print(posteriorSkin[cY, cX])
            if likeApple > likeNonApple:
                posteriorSkin[cY, cX] = 1
            else:
                posteriorSkin[cY, cX] = 0

            # print(likeSkin)
            # print(likeNonSkin)
    posteriorSkin = (posteriorSkin * 255).astype(np.uint8)
    TestImgs.append(posteriorSkin)

results_dir = 'results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# save the images
for i, img in enumerate(TestImgs):
    save_path = os.path.join(results_dir, f'posteriorSkin_{i}.png')
    imageio.imsave(save_path, img)
    print(f'Saved: {save_path}')

Processing Row  0 

Processing Row  1 

Processing Row  2 

Processing Row  3 

Processing Row  4 

Processing Row  5 

Processing Row  6 

Processing Row  7 

Processing Row  8 

Processing Row  9 

Processing Row  10 

Processing Row  11 

Processing Row  12 

Processing Row  13 

Processing Row  14 

Processing Row  15 

Processing Row  16 

Processing Row  17 

Processing Row  18 

Processing Row  19 

Processing Row  20 

Processing Row  21 

Processing Row  22 

Processing Row  23 

Processing Row  24 

Processing Row  25 

Processing Row  26 

Processing Row  27 

Processing Row  28 

Processing Row  29 

Processing Row  30 

Processing Row  31 

Processing Row  32 

Processing Row  33 

Processing Row  34 

Processing Row  35 

Processing Row  36 

Processing Row  37 

Processing Row  38 

Processing Row  39 

Processing Row  40 

Processing Row  41 

Processing Row  42 

Processing Row  43 

Processing Row  44 

Processing Row  45 

Processing Row  46 

Processing Row  47 

Pr